In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str,    
    "sr_tipo_archivo": str,
    "sr_id_archivo": str,
    "sr_subproceso": str
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

file_manager = FileManager(err_repo_path=conf.err_repo_path)

In [0]:
#Se ejecuta el UPDATE antes de inciar el proceso en caso de reproceso

statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_UPDATE.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_000.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso,
)

db.write_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}"))

In [0]:
#Se eliminan los registros de la tabla auxiliar antes de iniciar
statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_DELETE_000.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_DATAUX.TTAFOTRAS_ETL_MARCA_DESMARCA_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}"), table_name, "default", "append")


In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_MERGE_000.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}")

#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_DELETE_000.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_002.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso,
)

db.write_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}"))

In [0]:
# Cargar tabla Delta en un DataFrame
df = spark.read.format("delta").table(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}")

# Mostrar los primeros 100 registros
df.show(100)

In [0]:
import os

#output_path = f"/NCI/RCTRAS/VIV/INF/{params.sr_folio}_Comparacion_Archivo_Rechazo.csv"
output_path = f"{params.sr_folio}_Comparacion_Archivo_Rechazo"

full_path = SETTINGS.GENERAL.EXTERNAL_LOCATION + SETTINGS.GENERAL.PATH_RCTRAS + '/' + SETTINGS.GENERAL.PATH_INF + '/' + os.path.basename(output_path) +'.csv'
# Genera el archivo y decide si calcular MD5 o no
file_manager.generar_archivo_ctindi(
   df_final=df,  # DataFrame que se va a guardar
   full_file_name=full_path,
   header=True,
   calcular_md5=False  # Cambia a False si no quieres calcular el MD5
)


In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}")

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_001.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso,
)

db.write_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}"))

In [0]:
#Se eliminan los registros de la tabla auxiliar antes de insertar
statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_DELETE_001.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_DATAUX.TTCRXGRAL_MARCA_DESMARCA_INFO_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}"), table_name, "default", "append")

In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_MERGE_001.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_SDM_COMPARATIVO_{params.sr_folio}")

#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "PATRIF_INCO_0020_SDM_COMPARATIVO_DELETE_001.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)